In [2]:
:dep linfa = "0.7.0"
:dep linfa-clustering = "0.7.0"
:dep linfa-nn = "0.7.0"
:dep ndarray = "0.15.6"
:dep nalgebra = "0.29.0"
:dep plotly = "0.8.4"
:dep polars = { version = "0.39.2", features = ["lazy", "dtype-categorical", "ndarray"] }
:dep rand = "0.8.5"

:dep helper-functions = { path = "../helper-functions/" }

In [3]:
use std::f64::consts::PI;

use nalgebra::Point2;

use plotly::{
    common::*,
    layout::*;
    Trace,
};

use polars::prelude::*;

use rand::distributions::{
    Distribution,
    Uniform,
};

use helper_functions::prelude::*;

In [4]:
let mut rng = rand::thread_rng();
let uniform = Uniform::new(0.0, 1.0);
let total = 1_000;

let mut points = Vec::new();

for _ in 0..total {
    for radius in [1.0, 2.0, 3.0].iter() {
        let angle = uniform.sample(&mut rng) * 2.0 * PI;
        let x = radius * angle.cos();
        let y = radius * angle.sin();
        points.push(Point2::new(x, y));
    }
}

let point_x_values = points.iter().map(|point| point.x).collect::<Vec<f64>>();
let point_y_values = points.iter().map(|point| point.y).collect::<Vec<f64>>();

let mut raw_dataset = DataFrame::new(vec![
    Series::new("x", &point_x_values),
    Series::new("y", &point_y_values)
]).unwrap();

raw_dataset

shape: (3_000, 2)
┌───────────┬───────────┐
│ x         ┆ y         │
│ ---       ┆ ---       │
│ f64       ┆ f64       │
╞═══════════╪═══════════╡
│ -0.928004 ┆ 0.372571  │
│ 0.531795  ┆ -1.928003 │
│ 0.119769  ┆ 2.997608  │
│ -0.798322 ┆ -0.602231 │
│ -0.853284 ┆ -1.808841 │
│ …         ┆ …         │
│ 1.09443   ┆ 1.673984  │
│ -0.061012 ┆ 2.99938   │
│ -0.087698 ┆ 0.996147  │
│ 1.765881  ┆ -0.93897  │
│ -2.864909 ┆ -0.890111 │
└───────────┴───────────┘

In [5]:
let x = "x";
let y = "y";
let group_column = None;
let trace = "scatter";
let opacity = 1.0;

let traces = get_traces(
    x,
    y,
    group_column,
    &raw_dataset,
    trace,
    opacity,
);

let layout = None;

show_plot(traces, layout);

![](plots/plot.png)

In [6]:
let dataset = create_linfa_dataset(
    vec!["x", "y"],
    None,
    vec!["x", ""],
    &raw_dataset,
);

In [7]:
let model = dbscan_model(20, 0.6);
let predictions = dbscan_predict(&dataset, &model);
predictions

shape: (3_000, 1)
┌─────────────┐
│ predictions │
│ ---         │
│ i32         │
╞═════════════╡
│ 0           │
│ 1           │
│ 2           │
│ 0           │
│ 1           │
│ …           │
│ 1           │
│ 2           │
│ 0           │
│ 1           │
│ 2           │
└─────────────┘

In [8]:
let final_dataframe = polars::functions::concat_df_horizontal(
    &[raw_dataset.clone(), predictions.clone()]
).unwrap()
.lazy()
.with_column(
    col("predictions")
        .cast(DataType::String)
        .cast(DataType::Categorical(None, CategoricalOrdering::default()))
)
.collect()
.unwrap();

final_dataframe

shape: (3_000, 3)
┌───────────┬───────────┬─────────────┐
│ x         ┆ y         ┆ predictions │
│ ---       ┆ ---       ┆ ---         │
│ f64       ┆ f64       ┆ cat         │
╞═══════════╪═══════════╪═════════════╡
│ -0.928004 ┆ 0.372571  ┆ 0           │
│ 0.531795  ┆ -1.928003 ┆ 1           │
│ 0.119769  ┆ 2.997608  ┆ 2           │
│ -0.798322 ┆ -0.602231 ┆ 0           │
│ -0.853284 ┆ -1.808841 ┆ 1           │
│ …         ┆ …         ┆ …           │
│ 1.09443   ┆ 1.673984  ┆ 1           │
│ -0.061012 ┆ 2.99938   ┆ 2           │
│ -0.087698 ┆ 0.996147  ┆ 0           │
│ 1.765881  ┆ -0.93897  ┆ 1           │
│ -2.864909 ┆ -0.890111 ┆ 2           │
└───────────┴───────────┴─────────────┘

In [9]:
let x = "x";
let y = "y";
let group_column = Some("predictions");
let trace = "scatter";
let opacity = 1.0;

let traces = get_traces(
    x,
    y,
    group_column,
    &final_dataframe,
    trace,
    opacity,
);

let layout = None;

show_plot(traces, layout);

![](plots/plot2.png)